In [1]:
import numpy as np
from numba import jit, njit, prange
from Cube import read_charges_refined, read_cube
from VDW import *
from scipy import optimize
import time

@njit(cache=True, parallel=True)
def in_interaction_belt(positions, _xyz, _vdw):
    output = np.zeros((size_x, size_y, size_z))

    for i in prange(size_x):
        for j in prange(size_y):
            for k in prange(size_z):

                distances = []
                r = _xyz[:, j, i, k]
                for n in range(num_atoms):
                    rr = distance(r, positions[n])
                    distances.append(rr / _vdw[n])

                rmin = min(distances)

                if not rmin >= 1.2:
                    output[i, j, k] = 0  # False
                elif not rmin <= 2.20:
                    output[i, j, k] = 0  # False
                else:
                    output[i, j, k] = 1  # True

    return output


@jit(nopython=True, cache=True)
def jit_columbic_np(a, b, c):
    b = np.asarray([[b[0]] * 13, [b[1]] * 13, [b[2]] * 13])
    difference = a - b
    diff_sqr = np.square(difference)
    sum_diff = np.sum(diff_sqr, axis=0)  # sum (dif x , dif y, dif z) = (sum)
    sqr_sum_diff = np.sqrt(sum_diff)
    columbic = c / sqr_sum_diff
    columbic = np.sum(columbic)
    return columbic


@njit(nopython=True, cache=True, parallel=True)
def calculate_coulombic_grid(_size_x, _size_y, _size_z, _xyz, _positions_np, _charges_np):
    output = np.zeros((_size_x, _size_y, _size_z))
    for i in prange(_size_x):
        for j in prange(_size_y):
            for k in prange(_size_z):
                r = _xyz[:, j, i, k]
                output[i, j, k] = jit_columbic_np(_positions_np, r, _charges_np)
    return output


@jit(nopython=True, cache=True)
def RMSE_in_kcal(output, ref):
    diff = output - ref
    rmse = np.sqrt(np.square(diff.flatten()).sum() / (size_x * size_y * size_z))
    return rmse * HARTREE_TO_KCAL


@jit(nopython=True, cache=True)
def distance(output, ref):
    diff = output - ref
    rmse = np.sqrt(np.square(diff.flatten()).sum())
    return rmse


@jit(nopython=True, cache=True)
def RMSE_in_kcal_in_belt(output, ref, belt):
    diff = np.multiply(output, belt) - np.multiply(ref, belt)
    rmse = np.sqrt(np.square(diff.flatten()).sum() / belt.sum())
    return rmse * HARTREE_TO_KCAL


@jit(nopython=True, cache=True)
def RMSE(output, ref):
    diff = output - ref
    rmse = np.sqrt(np.square(diff.flatten()).sum() / (size_x * size_y * size_z))
    return rmse * HARTREE_TO_KCAL

#  MDCM Charges location
charges_path = "charges/butadiene/13-charges/13_charges_refined.xyz"
pos_charges_np = read_charges_refined(charges_path)

pcube = "cubes/butadiene/scan_extract_1.xyz.chk.fchk.pot.cube"
pcube_data, pcube_meta = read_cube(pcube)
pcube_atoms = pcube_meta["atoms"]
org = list(pcube_meta["org"])
xvec = list(pcube_meta["xvec"])[0]
yvec = list(pcube_meta["yvec"])[1]
zvec = list(pcube_meta["zvec"])[2]

positions_np = pos_charges_np[:, 0:3]
positions_np = positions_np.T
print(positions_np)
charges_np = pos_charges_np[:, -1]

size_x = pcube_data.shape[0]
size_y = pcube_data.shape[1]
size_z = pcube_data.shape[2]

x_values = np.arange(org[0], org[0] + xvec * size_x, xvec)
y_values = np.arange(org[1], org[1] + yvec * size_y, yvec)
z_values = np.arange(org[2], org[2] + zvec * size_z, zvec)

xx, yy, zz = np.meshgrid(x_values, y_values, z_values, indexing="ij")
xyz = np.array(np.meshgrid(x_values, y_values, z_values, indexing="xy"))

XYZ = np.column_stack((xx.ravel(), yy.ravel(), zz.ravel()))
XYZ = XYZ.T


read_map = [np.fromiter(x[1], dtype=np.float) for x in pcube_atoms]
atom_posistions = [x[1:] for x in read_map]
atom_posistions = np.array(atom_posistions, dtype=np.float64)
num_atoms = len(atom_posistions)
atoms = [int(x[0]) for x in read_map]
known_VDW = [VDWs[x] for x in atoms]
known_VDW = np.array(known_VDW, dtype=np.float64)
interaction_belt = in_interaction_belt(atom_posistions, xyz, known_VDW)


output = calculate_coulombic_grid(size_x, size_y, size_z, xyz, positions_np, charges_np)
base_error = RMSE_in_kcal_in_belt(output, pcube_data, interaction_belt)
print("Base Error: Testing RMSE() on my cube, ref. pcube", base_error)



_ColormakerRegistry()

[[ 2.49413876e-01  2.60275626e-01  3.76694024e+00  1.48414550e+00
   9.38749955e-01  1.70378123e+00 -1.49134667e+00 -2.49258748e+00
  -1.24352765e+00 -1.03418144e+00  1.59508338e+00 -3.90032295e+00
  -1.63905744e+00]
 [ 1.69579596e+00  3.63139848e-01  1.68375448e+00  2.28141146e+00
   1.19910368e+00  4.38828335e+00  2.90846953e+00  2.03197042e+00
  -2.15126574e+00 -1.09506104e+00 -2.83637981e+00 -1.61106618e+00
  -4.58371355e+00]
 [-1.33327152e-05 -9.37190288e-06  3.56178279e-03 -1.20057167e-02
   1.79162830e-03 -1.39515116e-02 -3.58250668e-03 -2.71718594e-01
  -3.88045163e-03 -1.39416078e-02 -2.46971025e-02  1.22063099e-02
   1.15535418e-03]]


C:\Users\Eric\Miniconda3\lib\site-packages\numba\core\decorators.py:255: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


Base Error: Testing RMSE() on my cube, ref. pcube 5.630115239688681


In [2]:
#  Loading previous weights
class out():
    def __init__(self):
        self.x =  [ 0.60267561,  4.33264136, -0.14129338,  3.61282246,  0.31147515,  0.8529962,  2.60622493, -2.42813974,  0.14948241, -0.85839552,  1.90305647,  0.06945667,  0.06303177,
         1.68092857,  0.10948372,  0.74777838,  4.38904829, -0.40672342, -3.16006587,  4.33175131, -0.35941405, -3.90201733,  2.44690258,  0.21462568,  1.10589301, -2.41577576,
         -0.03041944, -0.02261647, -1.04405489,  0.12538961, 2.96434675, -4.3711234,  -0.41999465, -2.98107655,  0.33195022,  0.65649628, -0.47436164,  -3.91128222, -0.57034649]
        
out = out()
     

def objective(x):
    """
    
    """
    for i in range(len(x)):
        positions_np[i%3, i//3] = x[i]
        
    output = calculate_coulombic_grid(size_x, size_y, size_z, xyz, positions_np, charges_np)
    output = RMSE_in_kcal_in_belt(output, pcube_data, interaction_belt)
    return output



#  Populate guesses
c = 0
x_guesses = []
for y in range(13):
    for x in range(3):
        x_guesses.append(positions_np[x, y])
        c += 1
        


out = optimize.minimize(objective, [*out.x], args=(), method='BFGS', 
                        jac=None, tol=None, callback=None, 
                        options={'gtol': 1e-05, 'norm': np.inf, 
                                 'eps': 1.4901161193847656e-05, 
                                 'maxiter': 1, 'disp': False, 
                                 'return_all': False})


print(out)


      fun: 0.5393653147703223
 hess_inv: array([[ 1.00548579e+00,  5.96896146e-03, -1.24046285e-03, ...,
         2.86917117e-03,  1.69206761e-03, -1.32761595e-02],
       [ 5.96896146e-03,  1.00634060e+00, -1.43757309e-03, ...,
         2.46054255e-03,  8.44133973e-04, -1.18750409e-02],
       [-1.24046285e-03, -1.43757309e-03,  1.00023041e+00, ...,
        -1.02584694e-03, -9.51035393e-04,  4.46758455e-03],
       ...,
       [ 2.86917117e-03,  2.46054255e-03, -1.02584694e-03, ...,
         9.98662217e-01, -3.39392586e-03,  4.08849459e-03],
       [ 1.69206761e-03,  8.44133973e-04, -9.51035393e-04, ...,
        -3.39392586e-03,  9.94071455e-01,  1.25360351e-02],
       [-1.32761595e-02, -1.18750409e-02,  4.46758455e-03, ...,
         4.08849459e-03,  1.25360351e-02,  9.89250433e-01]])
      jac: array([ 7.01723099e-05,  6.58384413e-05, -2.18623877e-05,  4.62497771e-06,
       -1.70709938e-05, -4.60536331e-05, -9.29374620e-05, -6.83159381e-06,
        6.01161718e-05,  7.42014199e-05, 

In [15]:
objective(out.x)

0.5393653147703223

In [16]:
print(out.x)

[ 0.60267544  4.33264115 -0.14129336  3.61282237  0.31147504  0.85299674
  2.60622543 -2.42814035  0.1494828  -0.85839441  1.90305783  0.06945802
  0.06303142  1.68092768  0.10948411  0.74777798  4.38904897 -0.40672367
 -3.16006594  4.33175157 -0.35941473 -3.90201733  2.44690213  0.21462539
  1.10589313 -2.41577388 -0.03042321 -0.02261637 -1.04405627  0.12539025
  2.96434686 -4.37112332 -0.41999421 -2.98107652  0.33194957  0.65649617
 -0.47436185 -3.91128245 -0.57034561]


In [17]:
print(positions_np)

[[ 0.60267544  3.61282237  2.60622543 -0.85839441  0.06303142  0.74777798
  -3.16006594 -3.90201733  1.10589313 -0.02261637  2.96434686 -2.98107652
  -0.47436185]
 [ 4.33264115  0.31147504 -2.42814035  1.90305783  1.68092768  4.38904897
   4.33175157  2.44690213 -2.41577388 -1.04405627 -4.37112332  0.33194957
  -3.91128245]
 [-0.14129336  0.85299674  0.1494828   0.06945802  0.10948411 -0.40672367
  -0.35941473  0.21462539 -0.03042321  0.12539025 -0.41999421  0.65649617
  -0.57034561]]


In [18]:
%matplotlib widget

pcube_0 = "cubes/butadiene/scan_extract_0.xyz.chk.fchk.pot.cube"
pcube_data_0, pcube_meta_0 = read_cube(pcube_0)
pcube_atoms_0 = pcube_meta_0["atoms"]
read_map_0 = [np.fromiter(x[1], dtype=np.float) for x in pcube_atoms_0]
atom_posistions_0 = [x[1:] for x in read_map_0]
atom_posistions_0 = np.array(atom_posistions_0, dtype=np.float64)

atom_x = []
atom_y = []
atom_z = []
for x in atom_posistions_0:
    atom_x.append(x[0])
    atom_y.append(x[1])
    atom_z.append(x[2])

print(atom_x, atom_y, atom_z)
import matplotlib.pyplot as plt
fig = plt.figure()
ax = plt.axes(projection='3d')
data = positions_np
ax.scatter3D(data[0], data[1], data[2], c=charges_np, cmap='bwr')
ax.scatter3D(atom_x, atom_y, atom_z, c='k', s=50)
for i in range(len(atoms)):
    ax.text(atom_x[i], atom_y[i], atom_z[i],  "{}".format(atoms[i]), color='g')
    


[2.09414, 0.0, 0.0, -2.09414, 2.009811, 3.95138, -1.835514, 1.835514, -3.95138, -2.00981] [2.783833, 1.372509, -1.372508, -2.783833, 4.824243, 1.92466, 2.288467, -2.288466, -1.924661, -4.824243] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [165]:
fig = plt.figure()
ax = plt.axes(projection='3d')

new_positions = np.zeros((3, 13))
for i in range(len(out.x)):
    new_positions[i%3, i//3] = out.x[i]

data = new_positions
ax.scatter3D(data[0], data[1], data[2], c=charges_np, cmap='bwr')
ax.scatter3D(atom_x, atom_y, atom_z, c='k', s=50)
for i in range(len(atoms)):
    ax.text(atom_x[i], atom_y[i], atom_z[i],  "{}".format(atoms[i]), color='g')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [140]:
print(atom_x, atom_y, atom_z)

[-1.128271, -1.128271, 1.128271, 1.128271, -2.862596, 0.613571, -2.91779, 2.91779, -0.613571, 2.862596] [3.233034, 0.795121, -0.795121, -3.233034, 4.305751, 4.22505, -0.148706, 0.148706, -4.22505, -4.305751] [-0.289852, 0.357642, 0.357642, -0.289852, -0.402338, -0.702448, 0.698045, 0.698045, -0.702448, -0.402338]


In [147]:
out.x

array([ 0.60267561,  4.33264136, -0.14129338,  3.61282246,  0.31147515,
        0.8529962 ,  2.60622493, -2.42813974,  0.14948241, -0.85839552,
        1.90305647,  0.06945667,  0.06303177,  1.68092857,  0.10948372,
        0.74777838,  4.38904829, -0.40672342, -3.16006587,  4.33175131,
       -0.35941405, -3.90201733,  2.44690258,  0.21462568,  1.10589301,
       -2.41577576, -0.03041944, -0.02261647, -1.04405489,  0.12538961,
        2.96434675, -4.3711234 , -0.41999465, -2.98107655,  0.33195022,
        0.65649628, -0.47436164, -3.91128222, -0.57034649])

In [148]:
data

array([[ 2.49413876e-01,  2.60275626e-01,  3.76694024e+00,
         1.48414550e+00,  9.38749955e-01,  1.70378123e+00,
        -1.49134667e+00, -2.49258748e+00, -1.24352765e+00,
        -1.03418144e+00,  1.59508338e+00, -3.90032295e+00,
        -1.63905744e+00],
       [ 1.69579596e+00,  3.63139848e-01,  1.68375448e+00,
         2.28141146e+00,  1.19910368e+00,  4.38828335e+00,
         2.90846953e+00,  2.03197042e+00, -2.15126574e+00,
        -1.09506104e+00, -2.83637981e+00, -1.61106618e+00,
        -4.58371355e+00],
       [-1.33327152e-05, -9.37190288e-06,  3.56178279e-03,
        -1.20057167e-02,  1.79162830e-03, -1.39515116e-02,
        -3.58250668e-03, -2.71718594e-01, -3.88045163e-03,
        -1.39416078e-02, -2.46971025e-02,  1.22063099e-02,
         1.15535418e-03]])